In [160]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

## Preparação dos dados

### Entendendo as relações entre as tabelas:

<img src="https://i.imgur.com/HRhd2Y0.png" width="600" height="400">

Através do esquema do BD disponibilizado pelo Kaggle, pode-se entender as relações entre as tabelas.

De acordo com a descrição do dataset no <a href="https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce?select=olist_orders_dataset.csv">kaggle</a>, a principal tabela `olist_orders_dataset` onde é possível obter as demais informações sobre o pedido.

#### Carregamento dos datasets

In [161]:
olist_orders_dataset = pd.read_csv(os.path.join('olist', 'olist_orders_dataset.csv'))
olist_order_items_dataset = pd.read_csv(os.path.join('olist', 'olist_order_items_dataset.csv'))
olist_customers_dataset = pd.read_csv(os.path.join('olist', 'olist_customers_dataset.csv'))
olist_products_dataset = pd.read_csv(os.path.join('olist', 'olist_products_dataset.csv'))
olist_sellers_dataset = pd.read_csv(os.path.join('olist', 'olist_sellers_dataset.csv'))
olist_order_payments_dataset = pd.read_csv(os.path.join('olist', 'olist_order_payments_dataset.csv'))
olist_order_reviews_dataset = pd.read_csv(os.path.join('olist', 'olist_order_reviews_dataset.csv'))
product_category_name_translation = pd.read_csv(os.path.join('olist', 'product_category_name_translation.csv'))
olist_geolocation_dataset = pd.read_csv(os.path.join('olist', 'olist_geolocation_dataset.csv'))

#### Implementação de uma função para verificação de nulidade e duplicidade

In [162]:
def verify_duplicate_null(df:pd.DataFrame)->tuple[bool, bool]:
    is_null = True if df.isnull().sum().sum() > 0 else False
    print(f'Colunas com valores nulos:\n{df.isnull().sum()}')
    is_duplicated =  True if df.duplicated().sum() > 0 else False
    print(f'Número de linhas duplicados: {df.duplicated().sum()}')
    return is_null, is_duplicated


### Vamos entender a tabela `olist_orders_dataset`

In [163]:
print(olist_orders_dataset.info())
verify_duplicate_null(olist_orders_dataset)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB
None
Colunas com valores nulos:
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
or

(True, False)

In [164]:
date_columns = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]
def transform_str_datetime(df:pd.DataFrame,cols:list)->pd.DataFrame:
    for col in cols:
        df[col] = pd.to_datetime(df[col])
    return df

olist_orders_dataset = transform_str_datetime(olist_orders_dataset,date_columns)

# Verify the conversion
olist_orders_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB


As linhas que contêm colunas nulas envolvem a dimensão temporal, o que é essencial para análise de dados. Então irei remover as linhas que possuam colunas nulas.

In [165]:
olist_orders_dataset = olist_orders_dataset.dropna()
olist_orders_dataset.info()


<class 'pandas.core.frame.DataFrame'>
Index: 96461 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       96461 non-null  object        
 1   customer_id                    96461 non-null  object        
 2   order_status                   96461 non-null  object        
 3   order_purchase_timestamp       96461 non-null  datetime64[ns]
 4   order_approved_at              96461 non-null  datetime64[ns]
 5   order_delivered_carrier_date   96461 non-null  datetime64[ns]
 6   order_delivered_customer_date  96461 non-null  datetime64[ns]
 7   order_estimated_delivery_date  96461 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.6+ MB


### Vamos agora verificar o `olist_order_items_dataset`

In [166]:
print(olist_order_items_dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB
None


In [167]:
verify_duplicate_null(olist_order_items_dataset)

Colunas com valores nulos:
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64
Número de linhas duplicados: 0


(False, False)

In [168]:
olist_order_items_dataset['shipping_limit_date'] = pd.to_datetime(olist_order_items_dataset['shipping_limit_date'])

### Verificação da tabela `olist_order_payments_dataset`

In [169]:
olist_order_payments_dataset.info()
verify_duplicate_null(olist_order_payments_dataset)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB
Colunas com valores nulos:
order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64
Número de linhas duplicados: 0


(False, False)

### Verificação da tabela `olist_order_reviews_dataset`


In [178]:
olist_order_reviews_dataset.info()
verify_duplicate_null(olist_order_reviews_dataset)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   review_id                99224 non-null  object        
 1   order_id                 99224 non-null  object        
 2   review_score             99224 non-null  int64         
 3   review_comment_title     99224 non-null  object        
 4   review_comment_message   99224 non-null  object        
 5   review_creation_date     99224 non-null  datetime64[ns]
 6   review_answer_timestamp  99224 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 5.3+ MB
Colunas com valores nulos:
review_id                  0
order_id                   0
review_score               0
review_comment_title       0
review_comment_message     0
review_creation_date       0
review_answer_timestamp    0
dtype: int64
Número de linhas duplicados: 0


(False, False)

Verificando as linhas com valores nulos.

In [171]:
#Os valores nulos serão transformados em valores vazios
olist_order_reviews_dataset = olist_order_reviews_dataset.fillna('')

transformando as colunas que contêm datas em timestamp

In [173]:
date_columns_reviews = [
    'review_creation_date',
    'review_answer_timestamp'
]

olist_order_reviews_dataset = transform_str_datetime(olist_order_reviews_dataset,date_columns_reviews)
olist_order_reviews_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   review_id                99224 non-null  object        
 1   order_id                 99224 non-null  object        
 2   review_score             99224 non-null  int64         
 3   review_comment_title     99224 non-null  object        
 4   review_comment_message   99224 non-null  object        
 5   review_creation_date     99224 non-null  datetime64[ns]
 6   review_answer_timestamp  99224 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 5.3+ MB


### Verificação da tabela `olist_products_dataset`

In [65]:
olist_products_dataset.info()
verify_duplicate_null(olist_products_dataset)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB
Colunas com valores nulos:
product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g

(True, False)

In [66]:
# Não faz sentido um produto não possuir categoria, então irei remover as linhas nulas
olist_products_dataset = olist_products_dataset.dropna()

### Verificação da tabela `olist_sellers_dataset`

In [174]:
olist_sellers_dataset.info()
verify_duplicate_null(olist_sellers_dataset)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB
Colunas com valores nulos:
seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64
Número de linhas duplicados: 0


(False, False)

### Verificação da tabela `olist_order_customer_dataset`

In [68]:
olist_customers_dataset.info()
verify_duplicate_null(olist_customers_dataset)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
Colunas com valores nulos:
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64
Número de linhas duplicados: 0


(False, False)

### Verificação da tabela `olist_geolocation_dataset`

In [69]:
olist_geolocation_dataset.info()
verify_duplicate_null(olist_geolocation_dataset)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  object 
 4   geolocation_state            1000163 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 38.2+ MB
Colunas com valores nulos:
geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_city               0
geolocation_state              0
dtype: int64
Número de linhas duplicados: 261831


(False, True)

Removendo as linhas duplicadas

In [70]:
olist_geolocation_dataset = olist_geolocation_dataset.drop_duplicates()
verify_duplicate_null(olist_geolocation_dataset)

Colunas com valores nulos:
geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_city               0
geolocation_state              0
dtype: int64
Número de linhas duplicados: 0


(False, False)

### Verificando integridade referencial

In [71]:
def check_referential_integrity(primary_df: pd.DataFrame, secondary_df: pd.DataFrame, key_columns = []) ->  set:
    # Find IDs that exist in secondary but not in primary
    if len(key_columns)==1:
        key_column = key_columns[0]
        orphaned_ids = set(secondary_df[key_column]) - set(primary_df[key_column])
    else:
        orphaned_ids = set(secondary_df[key_columns[1]]) - set(primary_df[key_columns[0]])
    print(f"Checking referential integrity for {key_columns[0]}")
    print(f"Total records in primary DataFrame: {len(primary_df)}")
    print(f"Total records in secondary DataFrame: {len(secondary_df)}")
    print(f"IDs that exist in secondary but not in primary: {len(orphaned_ids)}")
    if len(orphaned_ids) > 0:
        print("Example of orphaned IDs:",list(orphaned_ids)[:3])
        return orphaned_ids
    return set()

In [72]:
check_referential_integrity(olist_customers_dataset,olist_orders_dataset,['customer_id'])

Checking referential integrity for customer_id
Total records in primary DataFrame: 99441
Total records in secondary DataFrame: 96461
IDs that exist in secondary but not in primary: 0


set()

Considerando que não há inconsistência entre `olist_customers_dataset` e `olist_orders_dataset`, o próximo passo é verificar com as outras tabelas.

In [73]:
ids_reviews_orders = check_referential_integrity(olist_orders_dataset, olist_order_reviews_dataset,['order_id'])

Checking referential integrity for order_id
Total records in primary DataFrame: 96461
Total records in secondary DataFrame: 99224
IDs that exist in secondary but not in primary: 2858
Example of orphaned IDs: ['34b3d6f518fb789c443e4ec00f9bb0ce', 'b31a540b397cb096c3c5252eab692c9a', '5b0ee19a3125fc083348baa76d816ed0']


Excluindo as reviews orfãs do `olist_order_reviews_dataset`

In [74]:
olist_order_reviews_dataset = olist_order_reviews_dataset[~olist_order_reviews_dataset['order_id'].isin(ids_reviews_orders)]
check_referential_integrity(olist_orders_dataset, olist_order_reviews_dataset,['order_id'])

Checking referential integrity for order_id
Total records in primary DataFrame: 96461
Total records in secondary DataFrame: 96344
IDs that exist in secondary but not in primary: 0


set()

Verificando inconsistências entre `olist_order_payments_dataset` e `olist_orders_dataset`

In [75]:
ids_payments_orders = check_referential_integrity(olist_orders_dataset, olist_order_payments_dataset,['order_id'])

Checking referential integrity for order_id
Total records in primary DataFrame: 96461
Total records in secondary DataFrame: 103886
IDs that exist in secondary but not in primary: 2980
Example of orphaned IDs: ['34b3d6f518fb789c443e4ec00f9bb0ce', 'b31a540b397cb096c3c5252eab692c9a', '5b0ee19a3125fc083348baa76d816ed0']


In [76]:
olist_order_payments_dataset = olist_order_payments_dataset[~olist_order_payments_dataset['order_id'].isin(ids_payments_orders)]
check_referential_integrity(olist_orders_dataset, olist_order_payments_dataset,['order_id'])

Checking referential integrity for order_id
Total records in primary DataFrame: 96461
Total records in secondary DataFrame: 100739
IDs that exist in secondary but not in primary: 0


set()

Verificando inconsistências entre `olist_order_items_dataset` e `olist_orders_dataset`

In [77]:
check_referential_integrity(olist_order_items_dataset,olist_orders_dataset,['order_id'])

Checking referential integrity for order_id
Total records in primary DataFrame: 112650
Total records in secondary DataFrame: 96461
IDs that exist in secondary but not in primary: 0


set()

Verificando inconsistências entre `olist_order_items_dataset` e `olist_products_dataset`

In [78]:
ids_items_product =  check_referential_integrity(olist_products_dataset,olist_order_items_dataset,['product_id'])

Checking referential integrity for product_id
Total records in primary DataFrame: 32340
Total records in secondary DataFrame: 112650
IDs that exist in secondary but not in primary: 611
Example of orphaned IDs: ['212b9c1a73834eeae7694906e168edc6', '1bd0e965122ea1485d8f7f0c6792ebe0', '2ddbeb76f134690c05636162f2558594']


In [79]:
olist_order_items_dataset = olist_order_items_dataset[~olist_order_items_dataset['product_id'].isin(ids_items_product)]
check_referential_integrity(olist_products_dataset,olist_order_items_dataset,['product_id'])

Checking referential integrity for product_id
Total records in primary DataFrame: 32340
Total records in secondary DataFrame: 111046
IDs that exist in secondary but not in primary: 0


set()

Verificando inconsistências entre `olist_order_items_dataset` e `olist_sellers_dataset`

In [80]:
check_referential_integrity(olist_sellers_dataset,olist_order_items_dataset,['seller_id'])

Checking referential integrity for seller_id
Total records in primary DataFrame: 3095
Total records in secondary DataFrame: 111046
IDs that exist in secondary but not in primary: 0


set()

Verificando inconsistências entre `olist_sellers_dataset` e `olist_geolocation_dataset`

In [81]:
print(olist_geolocation_dataset.columns)
print(olist_sellers_dataset.columns)

Index(['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng',
       'geolocation_city', 'geolocation_state'],
      dtype='object')
Index(['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state'], dtype='object')


In [82]:
ids_zip_code = check_referential_integrity(olist_geolocation_dataset,olist_sellers_dataset,['geolocation_zip_code_prefix','seller_zip_code_prefix'])

Checking referential integrity for geolocation_zip_code_prefix
Total records in primary DataFrame: 738332
Total records in secondary DataFrame: 3095
IDs that exist in secondary but not in primary: 7
Example of orphaned IDs: [72580, 37708, 2285]


In [83]:
olist_sellers_dataset = olist_sellers_dataset[~olist_sellers_dataset['seller_zip_code_prefix'].isin(ids_zip_code)]
check_referential_integrity(olist_geolocation_dataset,olist_sellers_dataset,['geolocation_zip_code_prefix','seller_zip_code_prefix'])

Checking referential integrity for geolocation_zip_code_prefix
Total records in primary DataFrame: 738332
Total records in secondary DataFrame: 3088
IDs that exist in secondary but not in primary: 0


set()

Verificando inconsistências entre  `olist_geolocation_dataset` e `olist_customers_dataset`

In [84]:
olist_customers_dataset.columns

Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state'],
      dtype='object')

In [85]:
ids_zip_code_customer = check_referential_integrity(olist_geolocation_dataset,olist_customers_dataset,['geolocation_zip_code_prefix','customer_zip_code_prefix'])

Checking referential integrity for geolocation_zip_code_prefix
Total records in primary DataFrame: 738332
Total records in secondary DataFrame: 99441
IDs that exist in secondary but not in primary: 157
Example of orphaned IDs: [28160, 56327, 75784]


In [86]:
olist_customers_dataset = olist_customers_dataset[~olist_customers_dataset['customer_zip_code_prefix'].isin(ids_zip_code_customer)]
check_referential_integrity(olist_geolocation_dataset,olist_customers_dataset,['geolocation_zip_code_prefix','customer_zip_code_prefix'])

Checking referential integrity for geolocation_zip_code_prefix
Total records in primary DataFrame: 738332
Total records in secondary DataFrame: 99163
IDs that exist in secondary but not in primary: 0


set()

### Relacionamentos entre tabelas

In [93]:
print(olist_orders_dataset.columns)
print(olist_order_reviews_dataset.columns)

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')
Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object')


In [177]:
df_orders = pd.merge(olist_orders_dataset,olist_order_reviews_dataset,
                    on='order_id',how='inner')
print(f'orders + review: {len(df_orders)}')
print(f'orders:{len(olist_orders_dataset)}')

orders + review: 96344
orders:96461
